In [1]:
import pandas as pd
data = pd.read_excel("../../../db/clean_orj_zaman.xlsx")
data.head()

,text,label
0,japon sahil istihkȃmlarında beyaz bayraklar da...,1945-1962
1,çekici bir tesiri olacağı muhakkaktır ittifak ...,1945-1962
2,iki partide ton arpa satılmıştı kudüse kilomet...,1945-1962
3,tomruk işinden şikȃyette bulunduğunu şikȃyeti ...,1945-1962
4,mevzua müteallik iş yaptıkları takdirde vesika...,1945-1962


In [2]:
data = data.dropna()

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1988 non-null   object
 1   label   1988 non-null   object
dtypes: object(2)
memory usage: 46.6+ KB


In [4]:
import gensim
review_text = data.text.apply(gensim.utils.simple_preprocess)
review_text

0       [japon, sahil, istihkȃmlarında, beyaz, bayrakl...
1       [çekici, bir, tesiri, olacağı, muhakkaktır, it...
2       [iki, partide, ton, arpa, satılmıştı, kudüse, ...
3       [tomruk, işinden, şikȃyette, bulunduğunu, şikȃ...
4       [mevzua, müteallik, iş, yaptıkları, takdirde, ...
                              ...                        
1995    [olması, ülkemizin, demokrasimizin, büyük, güv...
1996    [yapıyor, mesela, reha, başında, türban, başör...
1997    [arazisi, üzerine, evini, yapıyor, yi, kaçak, ...
1998    [bildirdi, bakanlığın, türkiye, adalet, bakanl...
1999    [destek, istedi, bunun, üzerine, mustafa, sarı...
Name: text, Length: 1988, dtype: object

In [5]:
from gensim.models.doc2vec import TaggedDocument
class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])
 
docLabels = data.label.tolist()
data_text = review_text.tolist()
sentences = TaggedDocumentIterator(data_text, docLabels)

In [6]:
model = gensim.models.Doc2Vec(
    vector_size=100,
    window=10,
    min_count=2,
    alpha=0.025,
    workers=11,
    epochs=20
)

In [7]:
model.build_vocab(sentences)

In [8]:
model.epochs

20

In [9]:
model.train(sentences,total_examples=model.corpus_count,epochs=model.epochs)

In [10]:
model.save("d2v_orj_zaman.doc2vec")